In [1]:
from trino.dbapi import connect

conn = connect(
    host="localhost",
    port=8082,
    user="admin"    # anything, required by Trino
)

cur = conn.cursor()

cur.execute("SHOW CATALOGS")
catalogs = cur.fetchall()
catalogs
print(catalogs)

# 2️⃣ Show all schemas to confirm
cur.execute("SHOW SCHEMAS FROM delta")
schemas = cur.fetchall()
print(schemas)

cur.execute("SHOW TABLES IN delta.hyb_bronze")
print(cur.fetchall())

[['delta'], ['hive'], ['system']]
[['_control'], ['bronze2'], ['default'], ['hyb_bronze'], ['hyb_silver'], ['information_schema']]
[['ecg_bronze'], ['sample_bronze']]


In [2]:
from trino.dbapi import connect

conn = connect(
    host="localhost",
    port=8082,
    user="admin"
)
cur = conn.cursor()

# 1) Create schema (no location allowed in Delta)
cur.execute("""
CREATE SCHEMA IF NOT EXISTS delta.hyb_bronze
"""
)
print("Schema ready.")

Schema ready.


In [4]:
cur.execute("""
CREATE TABLE IF NOT EXISTS delta.hyb_bronze.ecg_bronze (
msg_timestamp BIGINT,
ecg DOUBLE,
annotation VARCHAR,
mqtt_topic VARCHAR,
received_time BIGINT
)
""")

cur.execute("DESCRIBE delta.hyb_bronze.ecg_bronze")
schema = cur.fetchall()

for col in schema:
    print(col)

['msg_timestamp', 'bigint', '', '']
['ecg', 'double', '', '']
['annotation', 'varchar', '', '']
['mqtt_topic', 'varchar', '', '']
['received_time', 'bigint', '', '']


In [15]:
cur.execute("SELECT * FROM delta.hyb_bronze.ecg_bronze")
df = cur.fetchall()
print(df)

[[1733472000000, 0.12, 'normal', 'ecg/topic', 1733472000500]]


In [17]:
import pandas as pd

cur.execute("SELECT * FROM delta.hyb_bronze.ecg_bronze")
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]

df = pd.DataFrame(rows, columns=cols)
print(df)

   msg_timestamp   ecg annotation mqtt_topic  received_time
0  1733472000000  0.12     normal  ecg/topic  1733472000500
